In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing

matplotlib.use("Agg")
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline


/Users/rickgentry/opt/anaconda3/envs/bdrl/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
config.DOW_30_TICKER

['AAPL',
 'MSFT',
 'JPM',
 'V',
 'RTX',
 'PG',
 'GS',
 'NKE',
 'DIS',
 'AXP',
 'HD',
 'INTC',
 'WMT',
 'IBM',
 'MRK',
 'UNH',
 'KO',
 'CAT',
 'TRV',
 'JNJ',
 'CVX',
 'MCD',
 'VZ',
 'CSCO',
 'XOM',
 'BA',
 'MMM',
 'PFE',
 'WBA',
 'DD']

## Fetch Data

In [4]:
# Loads numerical data from csv if filepath is specified, otherwise downloads it from yfinance for specified dates and tickers
def get_numerical_data(filepath='',start_date='2020-01-01',end_date='2021-01-01',ticker_list=config.DOW_30_TICKER):
    if filepath:
        df = data.load_dataset(filepath)
    else:
        df = YahooDownloader(start_date=start_date,end_date=end_date,ticker_list=ticker_list).fetch_data()
    return df

In [5]:
# Need to pull textual data from sources
def get_textual_data():
    pass

# Run through sentiment analysis model to get the sentiment
def analyze_textual_data():
    pass

# Compute sentiment score. This needs to be computed for every ticker and day based on the sentiment analysis models output for text related to that day.
def compute_score():
    pass

In [6]:
numerical_df = get_numerical_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
numerical_df.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-02,74.059998,75.150002,73.797501,74.333511,135480400,AAPL,3
1,2020-01-02,124.660004,126.269997,124.230003,123.267235,2708000,AXP,3
2,2020-01-02,328.549988,333.350006,327.700012,331.348572,4544400,BA,3
3,2020-01-02,149.000000,150.550003,147.979996,144.700500,3311900,CAT,3
4,2020-01-02,48.060001,48.419998,47.880001,46.443089,16708100,CSCO,3


In [84]:
numerical_df[-4:]['date']

7586    2020-12-31
7587    2020-12-31
7588    2020-12-31
7589    2020-12-31
Name: date, dtype: object

In [24]:
# time_fmt = "%Y-%m-%d"
# dates = pd.date_range('2020-01-01','2021-01-01').to_pydatetime()
# dates = np.array([datetime.strftime(r,time_fmt) for r in dates])
# tickers = np.array(config.DOW_30_TICKER)
# data = np.array(np.meshgrid(dates,tickers)).T.reshape(-1,2)

In [8]:
from datetime import datetime

def generate_sentiment_scores(start_date,end_date,tickers=config.DOW_30_TICKER,time_fmt="%Y-%m-%d"):
    dates = pd.date_range(start_date,end_date).to_pydatetime()
    dates = np.array([datetime.strftime(r,time_fmt) for r in dates])
    data = np.array(np.meshgrid(dates,tickers)).T.reshape(-1,2)
    scores = np.random.uniform(low=-1.0,high=1.0,size=(len(data),1))
    data = np.concatenate((data,scores),axis=1)
    df = pd.DataFrame(data,columns=['date','tic','sentiment'])
    return df


In [35]:
sentiment_df = generate_sentiment_scores('2020-01-02','2021-01-01')

In [47]:
sentiment_df[:61]

,date,tic,sentiment
0,2020-01-02,AAPL,0.2833926324598415
1,2020-01-02,MSFT,0.35161072063542154
2,2020-01-02,JPM,-0.5808555250854983
3,2020-01-02,V,-0.39750294456355584
4,2020-01-02,RTX,-0.2170646506354239
...,...,...,...
56,2020-01-03,MMM,0.05423965692744859
57,2020-01-03,PFE,0.2665994090757775
58,2020-01-03,WBA,-0.8667574036009138
59,2020-01-03,DD,0.27551254408626114


## Preprocess data

In [11]:
def join_data(numerical_df,sentiment_df):
    return numerical_df.merge(sentiment_df,on=['date','tic'])


In [12]:
def preprocess_data(numerical_df,sentiment_df,use_turbulence=False):
    fe = FeatureEngineer(use_turbulence=use_turbulence)
    numerical_df = fe.preprocess_data(numerical_df)
    df = join_data(numerical_df,sentiment_df)
    return df

In [50]:
# Single sample

two_day_numerical = numerical_df.loc[:60]
two_day_sentiment = sentiment_df.loc[:60]
two_day_data = preprocess_data(single_day_numerical,single_day_sentiment)

Successfully added technical indicators


In [52]:
two_day_data

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,sentiment
0,2020-01-02,74.059998,75.150002,73.797501,74.333511,135480400,AAPL,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,74.333511,74.333511,0.2833926324598415
1,2020-01-02,124.660004,126.269997,124.230003,123.267235,2708000,AXP,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,123.267235,123.267235,-0.11115165541267769
2,2020-01-02,328.549988,333.350006,327.700012,331.348572,4544400,BA,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,331.348572,331.348572,0.29541394800696064
3,2020-01-02,149.000000,150.550003,147.979996,144.700500,3311900,CAT,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,144.700500,144.700500,0.8798273908339771
4,2020-01-02,48.060001,48.419998,47.880001,46.443089,16708100,CSCO,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,46.443089,46.443089,-0.07381205009223857
5,2020-01-02,120.809998,121.629997,120.769997,113.316681,5205000,CVX,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,113.316681,113.316681,-0.4464516834557266
6,2020-01-02,64.800003,65.160004,63.480000,61.819908,5967300,DD,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,61.819908,61.819908,-0.957457220229198
7,2020-01-02,145.289993,148.199997,145.100006,148.199997,9502100,DIS,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,148.199997,148.199997,0.7801606410796214
8,2020-01-02,231.000000,234.639999,230.160004,227.913971,3736300,GS,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,227.913971,227.913971,-0.9926437559956203
9,2020-01-02,219.080002,219.759995,217.839996,213.260651,3935700,HD,3,0.000000,74.994187,72.950164,0.0,-66.666667,100.0,213.260651,213.260651,-0.7517608373429434


In [14]:
processed = preprocess_data(numerical_df,sentiment_df)

Successfully added technical indicators


In [17]:
# import itertools
# list_ticker = processed["tic"].unique().tolist()
# list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
# combination = list(itertools.product(list_date,list_ticker))

# processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
# processed_full = processed_full[processed_full['date'].isin(processed['date'])]
# processed_full = processed_full.sort_values(['date','tic'])


# processed_full = processed_full.fillna(0)

In [24]:
processed.sample(5)

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,sentiment
3478,2020-06-17,119.860001,120.129997,118.400002,117.619827,6722300,WMT,2,-1.290810,125.199126,116.131516,46.609719,-138.407024,18.794410,121.310914,121.087573,-0.398418752383233
3599,2020-06-23,46.900002,47.220001,46.500000,43.970928,18916600,XOM,1,0.368817,50.564067,40.015104,50.305326,0.772453,4.928755,44.004824,41.655123,-0.8059696803289789
5921,2020-10-13,54.270000,54.290001,53.619999,53.118870,20005800,INTC,1,0.846364,53.585163,47.605143,54.869043,165.159351,33.459339,50.166393,49.776057,0.18158025813670298
1441,2020-03-12,87.660004,89.610001,81.809998,81.815742,12206600,AXP,3,-8.979919,145.336436,84.050319,27.036437,-198.757983,66.161600,119.648064,122.248775,-0.9747474568266239
4890,2020-08-25,124.697502,125.180000,123.052498,124.424088,211495600,AAPL,1,6.845802,128.234868,96.448479,73.723113,127.021210,68.616624,106.625726,97.585338,0.3361142593207198


## Setting up the environment

In [162]:
trade = data_split(processed, '2020-12-01','2021-01-01')

In [147]:
last_df = trade.loc[21]

trade = trade.drop(21)

In [170]:
trade[trade.date == '2020-12-02']

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,sentiment
1,2020-12-02,122.019997,123.370003,120.889999,122.896355,89004200,AAPL,2,1.155794,122.805722,113.112443,56.727828,154.383004,18.326964,116.314661,115.369485,-0.984651738354918
1,2020-12-02,119.279999,122.849998,118.900002,121.537247,3271900,AXP,2,4.855969,128.749669,97.409153,62.028803,89.511153,27.531221,107.265336,104.394359,-0.9176770257026572
1,2020-12-02,213.009995,224.990005,210.300003,223.850006,25912300,BA,2,13.984483,239.745046,151.577955,63.765725,106.310809,46.270265,182.422001,172.716168,-0.9572068126980808
1,2020-12-02,173.259995,174.419998,172.279999,172.171539,1971000,CAT,2,3.476263,179.658458,158.813556,58.681287,64.315213,17.939864,166.248767,158.887985,0.04679674467070649
1,2020-12-02,43.389999,43.959999,43.349998,43.227016,17422200,CSCO,2,1.260845,44.596100,35.715147,61.543815,122.453264,49.933659,38.954216,38.811192,0.21756488479904545
1,2020-12-02,87.260002,91.309998,87.099998,88.617355,10509600,CVX,2,4.274846,97.089123,68.545152,59.083672,82.819903,25.952298,78.091903,75.165402,-0.5083814301305061
1,2020-12-02,63.480000,64.290001,63.250000,63.684902,5959500,DD,2,1.451896,65.811749,57.838978,58.141855,85.679606,31.433742,60.476208,58.747960,0.2728045113638675
1,2020-12-02,149.490005,154.009995,148.339996,153.610001,10601900,DIS,2,5.910297,157.873490,125.776510,64.685415,109.627809,52.602178,135.727333,131.107500,0.9273346290420486
1,2020-12-02,232.080002,238.130005,231.580002,236.720154,2136300,GS,2,8.360214,243.926541,196.584662,64.374630,110.967327,43.734589,211.794794,205.571125,0.6197933298098626
1,2020-12-02,273.970001,274.109985,269.570007,269.411774,4168600,HD,2,-0.944513,281.791293,263.140854,48.497968,-63.312473,23.109887,272.434312,274.191360,0.4380411540759901


In [168]:
trade[trade.date > '2020-12-02'][['date','open']]

,date,open
2,2020-12-03,123.519997
2,2020-12-03,122.849998
2,2020-12-03,228.300003
2,2020-12-03,173.869995
2,2020-12-03,43.779999
...,...,...
21,2020-12-31,218.399994
21,2020-12-31,58.060001
21,2020-12-31,39.330002
21,2020-12-31,144.199997


In [151]:
trade.loc[trade.index[0]+1:]


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,sentiment
1,2020-12-02,122.019997,123.370003,120.889999,122.896355,89004200,AAPL,2,1.155794,122.805722,113.112443,56.727828,154.383004,18.326964,116.314661,115.369485,-0.984651738354918
1,2020-12-02,119.279999,122.849998,118.900002,121.537247,3271900,AXP,2,4.855969,128.749669,97.409153,62.028803,89.511153,27.531221,107.265336,104.394359,-0.9176770257026572
1,2020-12-02,213.009995,224.990005,210.300003,223.850006,25912300,BA,2,13.984483,239.745046,151.577955,63.765725,106.310809,46.270265,182.422001,172.716168,-0.9572068126980808
1,2020-12-02,173.259995,174.419998,172.279999,172.171539,1971000,CAT,2,3.476263,179.658458,158.813556,58.681287,64.315213,17.939864,166.248767,158.887985,0.04679674467070649
1,2020-12-02,43.389999,43.959999,43.349998,43.227016,17422200,CSCO,2,1.260845,44.596100,35.715147,61.543815,122.453264,49.933659,38.954216,38.811192,0.21756488479904545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2020-12-30,216.000000,220.389999,215.649994,218.021530,8875100,V,2,1.774089,216.171158,203.211760,59.209227,263.789704,29.424279,209.395252,204.046652,0.02611872782509783
20,2020-12-30,58.830002,58.939999,58.060001,56.911888,18259800,VZ,2,-0.390335,61.048898,56.699199,43.708731,-189.807648,17.673111,58.962963,58.125952,0.08300506391087992
20,2020-12-30,39.520000,39.730000,39.200001,38.968510,4194300,WBA,2,0.022185,42.954975,38.082485,50.214539,-34.466659,3.350135,39.692671,38.252223,0.8986461664723171
20,2020-12-30,144.880005,145.149994,143.940002,143.580521,6250400,WMT,2,-0.837518,149.407225,141.840954,49.369350,-96.017810,5.379552,147.089382,144.891253,-0.6932360295904421


In [54]:
indicator_list = config.TECHNICAL_INDICATORS_LIST + ['sentiment']

In [55]:
stock_dimension = len(processed_full.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicator_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 30, State Space: 331


In [60]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
    
}

e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
env_trade, _ = e_trade_gym.get_sb_env()

In [75]:
e_trade_gym.data

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,sentiment
0,2020-12-01,121.010002,123.470001,120.010002,122.536896,128166800,AAPL,1,0.806734,122.766675,111.870384,56.465653,158.903167,18.326964,116.122535,115.195506,-0.6353855836609985
0,2020-12-01,120.320000,122.570000,119.849998,119.152794,3584400,AXP,1,4.740563,128.735444,94.836253,60.587597,92.037957,31.349381,106.629814,104.078234,0.6005440884823487
0,2020-12-01,214.309998,218.089996,213.000000,213.009995,15805200,BA,1,13.502710,237.931128,146.371872,61.131542,100.686039,41.044077,180.535001,171.670001,0.4103206537661357
0,2020-12-01,175.389999,176.570007,172.940002,171.567505,2710200,CAT,1,3.661695,179.348927,158.511125,58.336805,76.050401,20.330658,166.053826,158.454731,-0.5389422727751916
0,2020-12-01,43.009998,44.070000,43.009998,42.882305,23948800,CSCO,1,1.188129,44.341458,35.259680,60.649269,130.290847,49.933659,38.800245,38.741337,-0.5830288155235768
0,2020-12-01,89.279999,89.709999,87.070000,86.231079,9915700,CVX,1,4.377220,97.004396,66.739570,57.242696,76.670547,21.597130,77.459857,74.967448,-0.01776743709505868
0,2020-12-01,64.839996,65.230003,63.330002,63.256752,4505200,DD,1,1.486600,65.660502,57.529655,57.490340,98.279086,32.254347,60.318391,58.625112,-0.5216685812161448
0,2020-12-01,149.570007,151.399994,149.000000,149.440002,8827800,DIS,1,5.628248,157.252273,123.438727,62.437060,103.344798,48.584574,134.772000,130.784000,-0.9259336746217759
0,2020-12-01,231.960007,234.869995,231.350006,231.171967,2581000,GS,1,8.068587,242.898599,193.549264,62.155696,104.800845,39.014700,210.773879,204.969028,-0.5188158197824897
0,2020-12-01,278.730011,278.950012,275.549988,273.386841,3944900,HD,1,-0.821353,281.941664,263.464095,50.789610,9.794276,10.655830,272.877826,274.136699,0.3704807831218777


## Trading

In [61]:
# Initialize agent
agent = DRLAgent(env_trade)

In [63]:
# Initialize Model
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [64]:
# Load or train
path_to_saved_model = ''
model_a2c.load(path_to_saved_model)